## Hello World

In [51]:
print("hellp world")

hellp world


Install Requirements ()

In [ ]:
!pip install autogen-agentchat autogen-ext[openai] -q
!pip install groq -q

In [52]:
import os
from getpass import getpass

tokenGROQ = getpass('Enter GROQ_API_KEY here: ')
os.environ["GROQ_API_KEY"] = tokenGROQ

Enter GROQ_API_KEY here: ··········


In [69]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
custom_model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)
assistant = AssistantAgent("assistant", model_client=custom_model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("DONE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="the pdf file is a little bit large, can you read it if i sent it to you completely, or you need it in small chunks?")
await Console(stream)

---------- user ----------
the pdf file is a little bit large, can you read it if i sent it to you completely, or you need it in small chunks?
---------- assistant ----------
I'm a large language model, I don't have the capability to directly receive or read files, including PDFs. I can only process text-based input. If you'd like to share the content of the PDF with me, you can try copying and pasting the text into this chat window. If the text is too long, you can break it up into smaller chunks and share them with me sequentially. I'll do my best to assist you.
Enter your response: DONE
---------- user_proxy ----------
DONE


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='the pdf file is a little bit large, can you read it if i sent it to you completely, or you need it in small chunks?', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=88, completion_tokens=92), content="I'm a large language model, I don't have the capability to directly receive or read files, including PDFs. I can only process text-based input. If you'd like to share the content of the PDF with me, you can try copying and pasting the text into this chat window. If the text is too long, you can break it up into smaller chunks and share them with me sequentially. I'll do my best to assist you.", type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, request_id='c8e6ef7d-a75f-4bee-ade2-5650170f4a66', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, content='DONE', type='TextMessage')], stop_rea

In [53]:
!pip install arxiv -q

In [56]:
import arxiv

# Construct the default API client.
client = arxiv.Client()

# Search for the 10 most recent articles matching the keyword "quantum."
search = arxiv.Search(
  query = "Multi-agent LLM systems",
  max_results = 1,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in client.results(search):
  print(f"Entry ID: {result.entry_id}")
  print(f"\nTitle: {result.title}")
  print(f"\nAuthors: {', '.join([a.name for a in result.authors])}")
  print(f"\nPublished: {result.published}")
  print(f"\nUpdated: {result.updated}")
  print(f"\nSummary: {result.summary}")
  print(f"\nComment: {result.comment}")
  print(f"\nJournal Reference: {result.journal_ref}")
  print(f"\nDOI: {result.doi}")
  print(f"\nPrimary Category: {result.primary_category}")
  print(f"\nCategories: {result.categories}")
  print(f"\nPDF URL: {result.pdf_url}\n")

Entry ID: http://arxiv.org/abs/2502.13145v1

Title: Multimodal Mamba: Decoder-only Multimodal State Space Model via Quadratic to Linear Distillation

Authors: Bencheng Liao, Hongyuan Tao, Qian Zhang, Tianheng Cheng, Yingyue Li, Haoran Yin, Wenyu Liu, Xinggang Wang

Published: 2025-02-18 18:59:57+00:00

Updated: 2025-02-18 18:59:57+00:00

Summary: Recent Multimodal Large Language Models (MLLMs) have achieved remarkable
performance but face deployment challenges due to their quadratic computational
complexity, growing Key-Value cache requirements, and reliance on separate
vision encoders. We propose mmMamba, a framework for developing
linear-complexity native multimodal state space models through progressive
distillation from existing MLLMs using moderate academic computational
resources. Our approach enables the direct conversion of trained decoder-only
MLLMs to linear-complexity architectures without requiring pre-trained
RNN-based LLM or vision encoders. We propose an seeding strategy

In [60]:
paper = next(arxiv.Client().results(arxiv.Search(id_list=["1605.08386v1"])))
print(paper.get_short_id())

1605.08386v1


In [62]:
!pip install PyPDF2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:00


In [64]:
import PyPDF2
import openai

# Step 1: Extract text from the PDF
pdf_path = "/content/downloaded-paper.pdf"  # Adjust the path as needed
paper_text = ""
with open(pdf_path, "rb") as f:
    reader = PyPDF2.PdfReader(f)
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            paper_text += page_text + "\n"

# Step 2: Split the text into manageable chunks
def chunk_text(text, max_length=2000):
    """Splits text into chunks of approximately max_length characters."""
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]


chunks = chunk_text(paper_text, max_length=2000)


In [68]:
print(chunks[1])

nderlying set Fcan be
Date: September 12, 2018.
2010Mathematics Subject Classiﬁcation. Primary: 05C81, Secondary: 37A25, 11P21.
Key words and phrases. Heat-bath random walks, sampling, lattice points, Markov b ases.
1
2 CAPRICE STANLEY AND TOBIAS WINDISCH
reached by a random walk that uses moves from M, whereas for the continuous version, a
random sampling from the unit sphere suﬃces. However, in man y situations where a Markov
basis is known, the heat-bath random walk is evidently fast. For instance, it was shown in [3]
that the heat-bath random walk on contingency tables mixes r apidly when the number of
columns is ﬁxed. To work around the connectedness issue, a discrete hit-and-run algorithm
was introduced in [1] for arbitrary ﬁnite sets F ⊂Zd. At each step in this random walk, a
subordinate and unrestricted random walk starts at the curr ent lattice point u∈ Fand uses
the unit vectors to collect a set of proposals S⊂Zd. The random walk then moves from u
to a random point in S∩F.
Ra